<a href="https://colab.research.google.com/github/Asigen93/deteksiayam_yolov8_and_deepsort/blob/main/beta5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# jalankan di Colab cell (prefix !)
!pip install -U pip
!pip install ultralytics roboflow opencv-python-headless matplotlib tqdm pandas
# TensorFlow dibutuhkan untuk export/validasi TFLite
!pip install -U "tensorflow>=2.11.0"
!pip install -U yolox-bytetrack
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 77.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 125.8 MB/s  0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [ultralytics]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="QIxsCi9aE9bKIfs7vXkg")  # paste API key Anda
# Ganti workspace/project/version sesuai Roboflow Anda:
# Anda bisa dapatkan nama project & versi dari URL Roboflow atau dari dashboard
project = rf.workspace().project("countingayam-om39k-jj99v")  # jika error, lihat catatan di bawah
dataset = project.version(1).download("yolov8-obb")  # pilihan export: "yolov8" atau "yolov5"
print("Dataset downloaded to:", dataset.location)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to countingayam-1 in yolov8-obb:: 100%|██████████| 2726/2726 [00:13<00:00, 200.14it/s]


Dataset downloaded to: /content/countingayam-1


In [3]:
import os
# sesuaikan path kalau Anda pakai dataset.location dari SDK, contohnya:
dataset_path = "/content/countingayam-1"   # ganti sesuai lokasi unzip / dataset.location
# kalau pakai SDK: dataset_path = dataset.location

for root, dirs, files in os.walk(dataset_path):
    print(root, "->", len([f for f in files if f.endswith(('.jpg','.png'))]), "images")
    # break supaya tidak print semua
    break

# Jika Roboflow sudah menyertakan data.yaml, gunakan itu.
!sed -n '1,200p' {dataset_path}/data.yaml

/content/countingayam-1 -> 0 images
train: train/images
val: valid/images
test: test/images

names: 
  0: ayam

In [4]:
print(dataset.location)

/content/countingayam-1


In [ ]:
from ultralytics import YOLO

# Load arsitektur YOLOv11 kecil (nano)
model = YOLO("yolov8m.yaml")

# Training dengan dataset ayam
model.train(
    data="countingayam-1/data.yaml",
    epochs=50,
    imgsz=960,          # ukuran input, bisa diubah misalnya 960
    batch=16,           # ukuran batch training
    device=0            # 0 = GPU, 'cpu' jika tanpa GPU
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=countingayam-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, k

In [ ]:
# Validasi
results = model.val(data="/content/countingayam-1/data.yaml")
print(results)

# Prediksi contoh gambar di val/images
model.predict(
    source="/content/countingayam-1/valid/images",
    save=True,
    conf=0.25,
    imgsz=960
)

In [ ]:
from IPython.display import Image, display

# Tampilkan confusion matrix
display(Image(filename="/content/runs/detect/train/confusion_matrix.png"))

In [ ]:
from IPython.display import Image, display

# Tampilkan confusion matrix
display(Image(filename="/content/runs/detect/train/results.png"))

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracker.byte_tracker import STrack

# --- Konfigurasi ByteTrack ---
class BYTETrackerArgs:
    track_thresh = 0.5       # threshold deteksi minimal
    track_buffer = 30        # berapa lama ID boleh hilang
    match_thresh = 0.8       # ambang kecocokan antar frame
    aspect_ratio_thresh = 3.0
    min_box_area = 10
    mot20 = False

# --- Fungsi counting ayam ---
def infer_and_count_video(video_path, weight_path, labels=['ayam'], out_video='out_count.mp4',
                          conf_thres=0.25, iou_thres=0.45):

    # Load model YOLOv11
    model = YOLO(weight_path)

    # Open video
    cap = cv2.VideoCapture(video_path)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_video, fourcc, fps, (w,h))

    # Inisialisasi ByteTrack
    tracker = BYTETracker(BYTETrackerArgs(), frame_rate=fps)
    total_count = 0
    counted_ids = set()

    # Garis counting (vertikal di tengah)
    line_x = int(w * 0.5)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Deteksi dengan YOLOv11
        results = model.predict(frame, imgsz=960, conf=conf_thres, iou=iou_thres, verbose=False)[0]

        detections = []
        for r in results.boxes:
            x1,y1,x2,y2 = r.xyxy[0].cpu().numpy()
            conf = float(r.conf.cpu().numpy())
            cls = int(r.cls.cpu().numpy())
            detections.append([x1, y1, x2-x1, y2-y1, conf, cls])  # format xywh

        # Update tracker
        online_targets = tracker.update(np.array(detections), (h, w), (h, w))

        for t in online_targets:
            tlwh = t.tlwh
            x1,y1,x2,y2 = int(tlwh[0]), int(tlwh[1]), int(tlwh[0]+tlwh[2]), int(tlwh[1]+tlwh[3])
            track_id = t.track_id
            cls_id = int(t.class_id) if hasattr(t, "class_id") else 0
            label = labels[cls_id] if cls_id < len(labels) else str(cls_id)

            # Gambar bounding box
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
            cv2.putText(frame, f"{label} ID:{track_id}", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 3)

            # Ambil centroid
            cx = int((x1+x2)/2)
            cy = int((y1+y2)/2)

            # Cek crossing line (kiri → kanan / kanan → kiri)
            if track_id not in counted_ids:
                if cx >= line_x-5 and cx <= line_x+5:  # buffer 5px
                    counted_ids.add(track_id)
                    total_count += 1

        # Gambar garis & total
        cv2.line(frame, (line_x,0), (line_x,h), (255,0,0), 3)
        cv2.putText(frame, f"Total count: {total_count}", (20,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 4)

        writer.write(frame)

    cap.release()
    writer.release()
    print("Selesai. Output:", out_video)


# --- Contoh panggil ---
infer_and_count_video(
    video_path='/content/tes1.mp4',
    weight_path='runs/detect/train/weights/best.pt',
    labels=['ayam'],
    out_video='result_count_bytetrack.mp4',
    conf_thres=0.25,
    iou_thres=0.2
)

In [ ]:
from ultralytics import YOLO

# path weight hasil training:
weight_path = "runs/detect/train/weights/best.pt"
model = YOLO(weight_path)

# 1) Export FP16 (smaller, default no int8):
model.export(format="tflite", imgsz=960, optimize=True)
# Hasil: file seperti best-fp16.tflite di working dir

# 2) (Opsional) Export INT8 (post-training quantization)
# Pastikan dataset/data.yaml ada, Ultralytics akan gunakan data untuk calibrasi quantization
try:
    model.export(format="tflite", imgsz=640, int8=True, data='data.yaml', optimize=True)
    print("INT8 export selesai.")
except Exception as e:
    print("INT8 export gagal, cek error:", e)
    print("Gunakan fallback FP16 atau lakukan quantization manual.")

In [ ]:
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="best-fp16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input:", input_details)
print("Output:", output_details)

# contoh preprocess satu image:
import cv2, numpy as np
img = cv2.imread('/content/some_test_image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = input_details[0]['shape'][1], input_details[0]['shape'][2]
img_resized = cv2.resize(img, (w,h))
img_arr = img_resized.astype('float32') / 255.0
img_arr = np.expand_dims(img_arr, axis=0)

interpreter.set_tensor(input_details[0]['index'], img_arr)
interpreter.invoke()
outs = [interpreter.get_tensor(o['index']) for o in output_details]
for o in outs:
    print("out shape:", o.shape)
# inspeksi array untuk tahu format keluaran; parsing custom diperlukan